In [3]:
import pandas as pd
import ast
from collections import Counter
import math
import re

In [4]:
# def get_bigram_corpus(wordlist):
#     """
#     Returns the bigram corpus given by the input [wordlist]
    
#     get_bigram_corpus creates a dictionary with bigrams from the [wordlist]
#     as keys and counts the instances of each bigram to assign values (except
#     for the bigram ('.', '<s>') which represents the start of one review and
#     the end of another)
    
#     wordlist: a list of words (strings)
#     """
#     corpus = {}
#     for i, word in enumerate(wordlist[1:], start=1):
#         if word != '<s>':
#             if (wordlist[i-1], word) not in corpus:
#                 corpus[(wordlist[i-1], word)] = 1
#             else:
#                 corpus[(wordlist[i-1], word)] += 1
#     return corpus

# def get_smooth_bigram_corpus(tokenlist, bigram_corpus):
#     """
#     Returns a dataframe object where the columns and rows are labeled with the tokens
#     in [tokenlist] and the elements are the counts of each bigram (defaulting to 1 to handle
#     add-one smoothing) in the format (row, column) so that the count for bigram (x, y) is 
#     found with df.loc[x, by]
    
#     get_smooth_bigram_corpus also appends the unknown word character [<UNK>] to handle 
#     unknown words
    
#     tokenlist: a list of tokens (strings)
#     bigram_corpus: a dictionary of bigram:count pairings
#     """
#     tokenlist.append('<UNK>')
#     df = pd.DataFrame(1, index = tokenlist, columns = tokenlist) 
#     for bigram in bigram_corpus:
#         df.loc[bigram[0], bigram[1]] += bigram_corpus[bigram]
#     return df

# def get_smooth_bigram_prob(bigram, smooth_bigram_corpus):
#     """
#     Returns the probability of a given [bigram] on a given [smooth_bigram_corpus]
    
#     get_smooth_bigram_prob takes the ratio of the value of [bigram] (df.loc(bigram[0], bigram[1]))
#     in the table [smooth_bigram_corpus] to the sum of all elements in the same row
    
#     bigram: a bigram (tuple of strings)
#     smooth_bigram_corpus: a dataframe with tokens as row and column names and bigram counts as values
#     """
#     return smooth_bigram_corpus.loc[bigram[0], bigram[1]] / smooth_bigram_corpus.loc[bigram[0]].sum()

In [5]:
train_df = pd.read_csv('./data_release/train.csv', encoding='cp1252')

In [7]:
train_df

,sentence,pos_seq,label_seq
0,Ca n't fail to be entertaining .,"['VERB', 'ADV', 'VERB', 'PART', 'VERB', 'ADJ',...","[0, 0, 0, 0, 0, 0, 0]"
1,How much was he going to tell her ?,"['ADV', 'ADJ', 'VERB', 'PRON', 'VERB', 'PART',...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"Up until that news hit the Committee , Don had...","['ADP', 'ADP', 'DET', 'NOUN', 'VERB', 'DET', '...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,Could go on to the rugby and go with them coul...,"['VERB', 'VERB', 'PART', 'ADP', 'DET', 'NOUN',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"Finally , we went to the office and they gave ...","['ADV', 'PUNCT', 'PRON', 'VERB', 'ADP', 'DET',...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
5,It must be shown that the defendant intended (...,"['PRON', 'VERB', 'VERB', 'VERB', 'ADP', 'DET',...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
6,The Englishman nodded and poured himself more ...,"['DET', 'PROPN', 'VERB', 'CCONJ', 'VERB', 'PRO...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,I demanded .,"['PRON', 'VERB', 'PUNCT']","[0, 0, 0]"
8,What is not known is information on the locati...,"['NOUN', 'VERB', 'ADV', 'VERB', 'VERB', 'NOUN'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,That 's getting in the,"['DET', 'VERB', 'VERB', 'ADP', 'DET']","[1, 0, 0, 0, 0]"


# HMM Model

In [92]:
# class HMM():
#     def __init__(self, training_df):
#         self.bigram_counts = {}
#         self.POS_counts = Counter()
        
#         self.lex_gen = {}
#         self.word_pattern = re.compile("(\w+|<s> |[,.!?;\(\)])")

#         self.training_df = training_df
            
#         for row in self.training_df.iterrows():
 
#             POS_list = ast.literal_eval(row[1][1])
#             POS_list.insert(0, '<START>')
            
#             POS_bigram_counts = get_bigram_corpus(POS_list)
            
#             sentence_list = row[1][0].split()
#             sentence_list.insert(0, '<s>')
            
            
#             for i, word in enumerate(sentence_list):
#                 if word not in self.lex_gen:
#                     self.lex_gen[word] = {POS_list[i] : 1}
#                 else:
#                     if POS_list[i] not in self.lex_gen[word]:
#                         self.lex_gen[word][POS_list[i]] = 1
#                     else:
#                         self.lex_gen[word][POS_list[i]] += 1
            
#             # Updates list of POS counts across whole training set 
#             self.POS_counts.update(POS_list)
            
#             for bigram in POS_bigram_counts: 
#                 if bigram in self.bigram_counts:
#                     self.bigram_counts[bigram] += POS_bigram_counts[bigram]
#                 else:
#                     self.bigram_counts[bigram] = POS_bigram_counts[bigram]
        
#         self.smoothed_bigram_counts_df = get_smooth_bigram_corpus(list(self.POS_counts), self.bigram_counts)
    


      
#     def prob_tagged_sequence(self, sequence):
#         '''
        
#         sequence: tuple where first element is sentence as a string, second element is a 
#             list of POS associated with each word of sentence 
#         '''
        
#         sentence_list = self.word_pattern.findall(sequence[0])
#         POS_list = sequence[1]
        
#         log_prob_acc = 0

#         for i, POS in enumerate(POS_list):
#             if i == 0:
#                 continue

#             POS_bigram = (POS_list[i-1], POS) 
#             log_prob_acc += math.log(get_smooth_bigram_prob(POS_bigram, self.smoothed_bigram_counts_df))

#             log_prob_acc += math.log(self.lex_gen[sentence_list[i]].get(POS, 1)/dict(self.POS_counts)[POS]) 
        
#         return math.exp(log_prob_acc)
    
    
#     def viterbi(self, sentence):
#         '''
#         sentence: string
#         '''
        
#         sentence_list = self.word_pattern.findall(sentence)

#         backpointers = []
#         scores = []
#         previous_scores = []

#         POS_list = list(self.POS_counts)
#         num_lex_categories = len(self.POS_counts)

#         #initialization
#         for i in range(1, num_lex_categories):
#             POS = POS_list[i]

#             initial_prob = get_smooth_bigram_prob(('<START>', POS), self.smoothed_bigram_counts_df)
#             lex_gen_prob = self.lex_gen.get(sentence_list[0], '<UNK>').get(POS, 1)/self.POS_counts.get(POS)

#             previous_scores.append(initial_prob * lex_gen_prob)

#         #iteration
#         for t in range(1, len(sentence_list)):

#             t_backpointers = []

#             for i in range(1, num_lex_categories):

#                 temp_backpointer = None
#                 max_score = (0, -1)

#                 for j in range(1, num_lex_categories):
#                     transition_prob = get_smooth_bigram_prob((POS_list[j], POS_list[i]), self.smoothed_bigram_counts_df) 
#                     lex_gen_prob = self.lex_gen.get(sentence_list[t], '<UNK>').get(POS, 1) / self.POS_counts.get(POS)
#                     #index j-1 accounts for exclusion of START POS during init of previous_scores
#                     score = previous_scores[j-1] * transition_prob * lex_gen_prob
#                     scores.append(score)
                    
#                     print(j)
#                     print(score)

#                     if score > max_score[0]:
#                         max_score = (score, j)
#                         temp_backpointer = j

#                 t_backpointers.append(temp_backpointer)
#                 previous_scores = scores

#             backpointers.append(t_backpointers)

#         #backtracking
#         tags = []
#         max_index = max_score[1]
#         print(backpointers)
#         for bptr_list in reversed(backpointers):
#             print(len(bptr_list))
#             print(max_index)
#             tags.insert(0, POS_list[max_index])
            
#             #index max_index-1 accounts for exclusion of START POS during init of backpointer lists
#             max_index = bptr_list[max_index-1]
            
#         return tags
            


In [ ]:
# def preprocess(sentence):
#     word_pattern = re.compile("(\w+|<s> |[,.!?;\(\)])")
#     return word_pattern.findall(sentence)

In [220]:
class HMM_Metaphor_Tagger():
    def __init__(self, training_data):
        self.tag_counts = Counter()
        self.tag_bigrams = {}
        self.emissions = {}
        
        #iterate through training examples
        for row in training_data.iterrows():
            
            #preprocess: add start characters and labels for computing initial probabilities
            tags_string = row[1][2]
            tags = ast.literal_eval(tags_string)
            tags.insert(0, '<START>')
            sentence = row[1][0].split()
            sentence.insert(0, '<s>')
            
            #get label bigram counts -- (0,0), (0,1), (1,0), (1,1), ('<START>',0), ('<START>',1)
            for t in range(1, len(tags)):
                tag_bigram = (tags[t-1], tags[t])
                if tag_bigram not in self.tag_bigrams:
                    self.tag_bigrams[tag_bigram] = 1
                else:
                    self.tag_bigrams[tag_bigram] += 1
                    
            #get individual tag counts
            self.tag_counts.update(tags)
            
            #get emission counts
            for i, word in enumerate(sentence):
                if word not in self.emissions:
                    self.emissions[word] = {tags[i] : 1}
                else:
                    if tags[i] not in self.emissions[word]:
                        self.emissions[word][tags[i]] = 1
                    else:
                        self.emissions[word][tags[i]] += 1

        print(self.tag_counts)
        print(self.tag_bigrams)

      
    def prob_tagged_sequence(self, sequence):
        '''
        sequence: tuple where first element is sentence as a string, second element is a 
            list of tags associated with each word of sentence 
            
        tags are binary (0 or 1)
        '''
        sentence = sequence[0].split()
        tags = sequence[1]
        
        #initialize log_prob_acc with initial probability
        initial_transition_prob = self.tag_bigrams[('<START>', tags[0])] / self.tag_counts['<START>']
        if sentence[0] in self.emissions:
            initial_emission_prob = self.emissions[sentence[0]].get(tags[0], 1) / self.tag_counts[tags[0]]
        else:
            initial_emission_prob = 1 / self.tag_counts[tags[0]]
        log_prob_acc = math.log(initial_transition_prob) + math.log(initial_emission_prob)
        
        #sum log transition and emission probabilities
        for t in range(1, len(tags)):
            tag_bigram = (tags[t-1], tags[t])
            transition_prob = self.tag_bigrams[tag_bigram] / self.tag_counts[tags[t-1]]
            if sentence[t] in self.emissions:
                emission_prob = self.emissions[sentence[t]].get(tags[t], 1) / self.tag_counts[tags[t]]
            else:
                emission_prob = 1 / self.tag_counts[tags[t]]
            log_prob_acc = math.log(transition_prob) + math.log(emission_prob)
        
        return math.exp(log_prob_acc)
    
    
    def viterbi(self, sentence):
        '''
        sentence: string where each token (punctuation included) is separated by a space
        '''
        sentence = sentence.split()

        backpointers = []
        scores = []
        previous_scores = []

        tags = list(self.tag_counts)

        #initialization
        for t in range(1, len(tags)):
            tag = tags[t]

            initial_transition_prob = self.tag_bigrams[('<START>', tag)] / self.tag_counts['<START>']
            if sentence[0] in self.emissions:
                initial_emission_prob = self.emissions[sentence[0]].get(tag, 1) / self.tag_counts[tag]
            else:
                initial_emission_prob = 1 / self.tag_counts[tag]
            
            previous_scores.append(initial_transition_prob * initial_emission_prob)
            
        #iteration
        #w is index of current word
        for w in range(1, len(sentence)):

            w_backpointers = []
            
            #t is index of current tag
            for t in range(1, len(tags)):
                
                t_backpointer = None
                max_score = (0, None)

                #j is index of previous tag
                for j in range(1, len(tags)):
                    
                    transition_prob = self.tag_bigrams[(tags[j], tags[t])] / self.tag_counts[tags[j]]
                    if sentence[w] in self.emissions:
                        emission_prob = self.emissions[sentence[w]].get(tags[t], 1) / self.tag_counts[tags[t]]
                    else:
                        emission_prob = 1 / self.tag_counts[tags[t]]
                    
                    score = previous_scores[j-1] * transition_prob * emission_prob                    
                    scores.append(score)
                    if score > max_score[0]:
                        max_score = (score, j)
                        t_backpointer = j

                w_backpointers.append(t_backpointer)

            previous_scores = scores
            backpointers.insert(0, w_backpointers)
            
        #backtracking
        max_index = max_score[1]
        output = [tags[max_index]]
        for bptrs in backpointers:
            max_index = bptrs[max_index-1]
            output.insert(0, tags[max_index])
            
        return output
            


In [221]:
model_1 = HMM_Metaphor_Tagger(train_df)
# hmm.prob_tagged_sequence(('the', [0]))
model_1.viterbi('the man is drowning in debt')

Counter({0: 103571, 1: 13051, '<START>': 6323})
{('<START>', 0): 5830, (0, 0): 87135, (0, 1): 10169, (1, 0): 10606, (1, 1): 2389, ('<START>', 1): 493}


[0, 0, 0, 0, 0, 0]